In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test_QoiMO9B.csv')
meal_info = pd.read_csv('../input/meal_info.csv')
center_info = pd.read_csv('../input/fulfilment_center_info.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
meal_info.head()

In [ ]:
center_info.head()

In [ ]:
train = train.merge(meal_info, on='meal_id')
train = train.merge(center_info, on='center_id')

test = test.merge(meal_info, on='meal_id')
test = test.merge(center_info, on='center_id')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isnull().sum(axis=0)

In [ ]:
sns.countplot(train['emailer_for_promotion'])

In [ ]:
sns.countplot(train['homepage_featured'])

In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(train['category'])

In [ ]:
sns.countplot(train['cuisine'])

In [ ]:
sns.countplot(train['week'])

In [ ]:
sns.countplot(train['city_code'])

In [ ]:
sns.countplot(train['region_code'])

In [ ]:
sns.countplot(train['center_type'])

In [ ]:
sns.distplot(np.log1p(train['op_area']))

In [ ]:
sns.distplot(np.log1p(train['num_orders']))

In [ ]:
sns.distplot(np.log1p(train['checkout_price']))

In [ ]:
sns.distplot(np.log1p(train['base_price']))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y = train['base_price'], x = train['num_orders'])
plt.xlabel('num_orders', fontsize=13)
plt.ylabel('base_price', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y = train['checkout_price'], x = train['num_orders'])
plt.xlabel('num_orders', fontsize=13)
plt.ylabel('checkout_price', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y = train['op_area'], x = train['num_orders'])
plt.xlabel('num_orders', fontsize=13)
plt.ylabel('op_area', fontsize=13)
plt.show()

In [ ]:
train['num_orders'].max()

In [ ]:
train = train.drop(train[(train['num_orders']>20000)].index)

In [ ]:
# train['diff_amount'] = train['checkout_price'] - train['base_price'] 
# test['diff_amount'] = test['checkout_price'] - test['base_price'] 

In [ ]:
#Correlation map to see how features are correlated with Num of orders
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
train['city_code'] = train['city_code'].astype(str)
train['region_code'] = train['region_code'].astype(str)

test['city_code'] = test['city_code'].astype(str)
test['region_code'] = test['region_code'].astype(str)

In [ ]:
train.head()

In [ ]:
train['checkout_price'] = np.log(train['checkout_price'])
train['base_price'] = np.log(train['base_price'])
#train['diff_amount'] = np.log(train['diff_amount'])

test['checkout_price'] = np.log(test['checkout_price'])
test['base_price'] = np.log(test['base_price'])
#test['diff_amount'] = np.log(test['diff_amount'])

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
feature_names = [x for x in train.columns if x not in ['num_orders','id','center_id','meal_id']]
target = np.log(train['num_orders'])

In [ ]:
train[feature_names].columns

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,RidgeCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,VotingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train[feature_names].values)
    rmse= np.sqrt(-cross_val_score(model, train[feature_names].values, target, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
ridge = RidgeCV(alphas = [0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6, 10, 30, 60])

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(random_state=1))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(random_state=3))

In [ ]:
model_xgb = xgb.XGBRegressor(silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression')

In [ ]:
model_catboost = CatBoostRegressor(eval_metric='RMSE',verbose=False,random_seed=42,depth=6,task_type='GPU')

In [ ]:
# score = rmsle_cv(LinearRegression())
# print("lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# score = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# score = rmsle_cv(model_lgb)
# print("lgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# score = rmsle_cv(model_catboost)
# print("catboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [ ]:
# model_catboost1 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=1000,random_seed=42,depth=6,task_type='GPU')
# model_catboost2 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=1500,random_seed=42,depth=8,task_type='GPU')
# model_catboost3 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=2000,random_seed=42,depth=10,task_type='GPU')
# model_catboost4 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=2500,random_seed=42,depth=2,task_type='GPU')
# model_catboost5 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=500,random_seed=42,depth=4,task_type='GPU')
# averaged_models = AveragingModels(models = (model_catboost1, model_catboost2, model_catboost3,model_catboost4, model_catboost5))

# score = rmsle_cv(averaged_models)
# print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
from catboost import CatBoostRegressor

model_catboost1 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=1000,random_seed=42,depth=6,task_type='GPU')
model_catboost1.fit(train[feature_names], target)
pred1 = model_catboost1.predict(test[feature_names])

model_catboost2 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=1500,random_seed=42,depth=8,task_type='GPU')
model_catboost2.fit(train[feature_names], target)
pred2 = model_catboost2.predict(test[feature_names])

model_catboost3 = CatBoostRegressor(eval_metric='RMSE',verbose=False,iterations=2000,random_seed=42,depth=10,task_type='GPU')
model_catboost3.fit(train[feature_names], target)
pred3 = model_catboost3.predict(test[feature_names])

In [ ]:
pred = (pred1 + pred2 + pred3)/3

In [ ]:
pred.min()

In [ ]:
## make submission
sub = pd.DataFrame()
sub['id'] = test['id']
sub['num_orders'] = np.exp(pred)
sub.to_csv('meal_output.csv', index=False)

In [ ]:
sub['num_orders'].max()